In [ ]:
---

_You are currently looking at **version 1.0** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-machine-learning/resources/bANLa) course resource._

---

## Assignment 4 - Understanding and Predicting Property Maintenance Fines

This assignment is based on a data challenge from the Michigan Data Science Team ([MDST](http://midas.umich.edu/mdst/)). 

The Michigan Data Science Team ([MDST](http://midas.umich.edu/mdst/)) and the Michigan Student Symposium for Interdisciplinary Statistical Sciences ([MSSISS](https://sites.lsa.umich.edu/mssiss/)) have partnered with the City of Detroit to help solve one of the most pressing problems facing Detroit - blight. [Blight violations](http://www.detroitmi.gov/How-Do-I/Report/Blight-Complaint-FAQs) are issued by the city to individuals who allow their properties to remain in a deteriorated condition. Every year, the city of Detroit issues millions of dollars in fines to residents and every year, many of these fines remain unpaid. Enforcing unpaid blight fines is a costly and tedious process, so the city wants to know: how can we increase blight ticket compliance?

The first step in answering this question is understanding when and why a resident might fail to comply with a blight ticket. This is where predictive modeling comes in. For this assignment, your task is to predict whether a given blight ticket will be paid on time.

All data for this assignment has been provided to us through the [Detroit Open Data Portal](https://data.detroitmi.gov/). **Only the data already included in your Coursera directory can be used for training the model for this assignment.** Nonetheless, we encourage you to look into data from other Detroit datasets to help inform feature creation and model selection. We recommend taking a look at the following related datasets:

* [Building Permits](https://data.detroitmi.gov/Property-Parcels/Building-Permits/xw2a-a7tf)
* [Trades Permits](https://data.detroitmi.gov/Property-Parcels/Trades-Permits/635b-dsgv)
* [Improve Detroit: Submitted Issues](https://data.detroitmi.gov/Government/Improve-Detroit-Submitted-Issues/fwz3-w3yn)
* [DPD: Citizen Complaints](https://data.detroitmi.gov/Public-Safety/DPD-Citizen-Complaints-2016/kahe-efs3)
* [Parcel Map](https://data.detroitmi.gov/Property-Parcels/Parcel-Map/fxkw-udwf)

___

We provide you with two data files for use in training and validating your models: train.csv and test.csv. Each row in these two files corresponds to a single blight ticket, and includes information about when, why, and to whom each ticket was issued. The target variable is compliance, which is True if the ticket was paid early, on time, or within one month of the hearing data, False if the ticket was paid after the hearing date or not at all, and Null if the violator was found not responsible. Compliance, as well as a handful of other variables that will not be available at test-time, are only included in train.csv.

Note: All tickets where the violators were found not responsible are not considered during evaluation. They are included in the training set as an additional source of data for visualization, and to enable unsupervised and semi-supervised approaches. However, they are not included in the test set.

<br>

**File descriptions** (Use only this data for training your model!)

    train.csv - the training set (all tickets issued 2004-2011)
    test.csv - the test set (all tickets issued 2012-2016)
    addresses.csv & latlons.csv - mapping from ticket id to addresses, and from addresses to lat/lon coordinates. 
     Note: misspelled addresses may be incorrectly geolocated.

<br>

**Data fields**

train.csv & test.csv

    ticket_id - unique identifier for tickets
    agency_name - Agency that issued the ticket
    inspector_name - Name of inspector that issued the ticket
    violator_name - Name of the person/organization that the ticket was issued to
    violation_street_number, violation_street_name, violation_zip_code - Address where the violation occurred
    mailing_address_str_number, mailing_address_str_name, city, state, zip_code, non_us_str_code, country - Mailing address of the violator
    ticket_issued_date - Date and time the ticket was issued
    hearing_date - Date and time the violator's hearing was scheduled
    violation_code, violation_description - Type of violation
    disposition - Judgment and judgement type
    fine_amount - Violation fine amount, excluding fees
    admin_fee - $20 fee assigned to responsible judgments
state_fee - $10 fee assigned to responsible judgments
    late_fee - 10% fee assigned to responsible judgments
    discount_amount - discount applied, if any
    clean_up_cost - DPW clean-up or graffiti removal cost
    judgment_amount - Sum of all fines and fees
    grafitti_status - Flag for graffiti violations
    
train.csv only

    payment_amount - Amount paid, if any
    payment_date - Date payment was made, if it was received
    payment_status - Current payment status as of Feb 1 2017
    balance_due - Fines and fees still owed
    collection_status - Flag for payments in collections
    compliance [target variable for prediction] 
     Null = Not responsible
     0 = Responsible, non-compliant
     1 = Responsible, compliant
    compliance_detail - More information on why each ticket was marked compliant or non-compliant


___

## Evaluation

Your predictions will be given as the probability that the corresponding blight ticket will be paid on time.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC). 

Your grade will be based on the AUC score computed for your classifier. A model which with an AUROC of 0.7 passes this assignment, over 0.75 will recieve full points.
___

For this assignment, create a function that trains a model to predict blight ticket compliance in Detroit using `train.csv`. Using this model, return a series of length 61001 with the data being the probability that each corresponding ticket from `test.csv` will be paid, and the index being the ticket_id.

Example:

    ticket_id
       284932    0.531842
       285362    0.401958
       285361    0.105928
       285338    0.018572
                 ...
       376499    0.208567
       376500    0.818759
       369851    0.018528
       Name: compliance, dtype: float32

In [36]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.neural_network import MLPClassifier


def blight_model():
    #loading datasets
    train=pd.read_csv('train.csv',encoding = 'ISO-8859-1' )
    test=pd.read_csv('test.csv')
           
    #cleaning features from data   
    #droping rows, where label is null, usually due to status "not responsible by disposition"
    train.dropna(axis=0,subset=['compliance'],inplace=True)
    
    #
    
    #transforming text columns to labels
    le=LabelEncoder()
    colnames=['agency_name','inspector_name','violator_name','violation_description','disposition','grafitti_status','violation_code']
    train[colnames]=train[colnames].fillna(value='unknown')
    test[colnames]=test[colnames].fillna(value='unknown')
    traintest=pd.concat([train,test])
    le.fit(trainrtest['agency_name'])
    train['agency_name']=le.transform(train['agency_name'])
    test['agency_name']=le.transform(test['agency_name'])
    le.fit(traintest['inspector_name'])
    train['inspector_name']=le.transform(train['inspector_name'])
    test['inspector_name']=le.transform(test['inspector_name'])
    le.fit(traintest['violator_name'])
    train['violator_name']=le.transform(train['violator_name'])
    test['violator_name']=le.transform(test['violator_name'])
    le.fit(traintest['violation_description'])
    train['violation_description']=le.transform(train['violation_description'])
    test['violation_description']=le.transform(test['violation_description'])
    le.fit(traintest['disposition'])
    train['disposition']=le.transform(train['disposition'])
    test['disposition']=le.transform(test['disposition'])
    le.fit(traintest['grafitti_status'])
    train['grafitti_status']=le.transform(train['grafitti_status'])
    test['grafitti_status']=le.transform(test['grafitti_status'])
    le.fit(traintest['violation_code'])
    train['violation_code']=le.transform(train['violation_code'])
    test['violation_code']=le.transform(test['violation_code'])
    
    #spliting dates to single features
    train['ticket_issued_date']=pd.to_datetime(train['ticket_issued_date']).fillna(method='ffill')
    train['hearing_date']=pd.to_datetime(train['hearing_date']).fillna(method='ffill')
    test['ticket_issued_date']=pd.to_datetime(test['ticket_issued_date']).fillna(method='ffill')
    test['hearing_date']=pd.to_datetime(test['hearing_date']).fillna(method='ffill')
    train['ticket_hour']=train['ticket_issued_date'].apply(lambda x :x.hour)
    train['hearing_hour']=train['hearing_date'].apply(lambda x :x.hour)
    test['ticket_hour']=test['ticket_issued_date'].apply(lambda x :x.hour)
    test['hearing_hour']=test['hearing_date'].apply(lambda x :x.hour)
    train['ticket_issued_date']=train['ticket_issued_date'].apply(lambda x :x.toordinal())
    train['hearing_date']=train['hearing_date'].apply(lambda x :x.toordinal())
    test['ticket_issued_date']=test['ticket_issued_date'].apply(lambda x :x.toordinal())
    test['hearing_date']=test['hearing_date'].apply(lambda x :x.toordinal())
    
    
    #transofrming address to lat lon    
    latlon=pd.read_csv('latlons.csv')
    address=pd.read_csv('addresses.csv')
    latlon.dropna(axis=0,inplace=True)
    addlatlon=address.merge(latlon,how='inner',on='address')
    addlatlon.drop('address',axis=1,inplace=True)
    train.drop(['violation_street_number', 'violation_street_name', 'violation_zip_code'],axis=1,inplace=True)
    test.drop(['violation_street_number', 'violation_street_name', 'violation_zip_code'],axis=1,inplace=True)
    train=train.merge(addlatlon, how='left',on='ticket_id')
    test=test.merge(addlatlon,how='left',on='ticket_id')
    train[['lat','lon']]=train[['lat','lon']].fillna(value=0)
    
    #droping mailing address
    mailing=['mailing_address_str_number', 'mailing_address_str_name', 'city','state', 'zip_code', 'non_us_str_code', 'country']
    train.drop(mailing,axis=1,inplace=True)
    test.drop(mailing,axis=1,inplace=True)
    
    #extracting label from dateset
    y_train=train['compliance']
    #preparing answer series with name and indexes
    answer=pd.Series(index=test['ticket_id'],dtype='float32',name='compliance')
    #droping features unavailable in test data
    train.drop(['payment_amount','payment_date','payment_status','balance_due','collection_status','compliance','compliance_detail'],axis=1,inplace=True)
    
    
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(train)
    X_test_scaled = scaler.transform(test)
    

    
    
    clf = MLPClassifier(hidden_layer_sizes = [100, 100], alpha = 5.0,
                   random_state = 0, solver='lbfgs').fit(X_train_scaled, y_train)
    proba_test=clf.predict_proba(test)
    answer=proba_test
    
    return answer

In [201]:
train=pd.read_csv('train.csv',encoding = 'ISO-8859-1' )
test=pd.read_csv('test.csv')

/opt/conda/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (11,12,31) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [202]:
#droping rows, where label is null, usually due to status "not responsible by disposition"
train.dropna(axis=0,subset=['compliance'],inplace=True)

In [203]:
#transforming text columns to labels
le=LabelEncoder()
colnames=['agency_name','inspector_name','violator_name','violation_description','disposition','grafitti_status','violation_code']
train[colnames]=train[colnames].fillna(value='unknown')
test[colnames]=test[colnames].fillna(value='unknown')
traintest=pd.concat([train,test])
le.fit(traintest['agency_name'])
train['agency_name']=le.transform(train['agency_name'])
test['agency_name']=le.transform(test['agency_name'])
le.fit(traintest['inspector_name'])
train['inspector_name']=le.transform(train['inspector_name'])
test['inspector_name']=le.transform(test['inspector_name'])
le.fit(traintest['violator_name'])
train['violator_name']=le.transform(train['violator_name'])
test['violator_name']=le.transform(test['violator_name'])
le.fit(traintest['violation_description'])
train['violation_description']=le.transform(train['violation_description'])
test['violation_description']=le.transform(test['violation_description'])
le.fit(traintest['disposition'])
train['disposition']=le.transform(train['disposition'])
test['disposition']=le.transform(test['disposition'])
le.fit(traintest['grafitti_status'])
train['grafitti_status']=le.transform(train['grafitti_status'])
test['grafitti_status']=le.transform(test['grafitti_status'])
le.fit(traintest['violation_code'])
train['violation_code']=le.transform(train['violation_code'])
test['violation_code']=le.transform(test['violation_code'])

In [212]:
test.head(3)

,ticket_id,agency_name,inspector_name,violator_name,ticket_issued_date,hearing_date,violation_code,violation_description,disposition,fine_amount,...,state_fee,late_fee,discount_amount,clean_up_cost,judgment_amount,grafitti_status,ticket_hour,hearing_hour,lat,lon
0,284932,1,80,36667,734506,734521,38,146,5,200.0,...,10.0,20.0,0.0,0.0,250.0,1,14,9,42.407581,-82.986642
1,285362,1,115,111487,734507,734539,57,0,5,1000.0,...,10.0,100.0,0.0,0.0,1130.0,1,9,9,42.426239,-83.238259
2,285361,1,115,111487,734507,734539,25,156,5,100.0,...,10.0,10.0,0.0,0.0,140.0,1,9,9,42.426239,-83.238259


In [211]:
train.head(3)

,ticket_id,agency_name,inspector_name,violator_name,ticket_issued_date,hearing_date,violation_code,violation_description,disposition,fine_amount,...,state_fee,late_fee,discount_amount,clean_up_cost,judgment_amount,grafitti_status,ticket_hour,hearing_hour,lat,lon
0,22056,0,172,52989,731656,732026,175,75,5,250.0,...,10.0,25.0,0.0,0.0,305.0,1,11,10,42.390729,-83.124268
1,27586,0,208,77939,731694,732072,98,51,6,750.0,...,10.0,75.0,0.0,0.0,855.0,1,12,13,42.326937,-83.135118
2,22046,0,172,59170,731702,732026,175,75,5,250.0,...,10.0,25.0,0.0,0.0,305.0,1,11,10,42.145257,-83.208233


In [206]:
#spliting dates to single features
train['ticket_issued_date']=pd.to_datetime(train['ticket_issued_date']).fillna(method='ffill')
train['hearing_date']=pd.to_datetime(train['hearing_date']).fillna(method='ffill')
test['ticket_issued_date']=pd.to_datetime(test['ticket_issued_date']).fillna(method='ffill')
test['hearing_date']=pd.to_datetime(test['hearing_date']).fillna(method='ffill')
train['ticket_hour']=train['ticket_issued_date'].apply(lambda x :x.hour)
train['hearing_hour']=train['hearing_date'].apply(lambda x :x.hour)
test['ticket_hour']=test['ticket_issued_date'].apply(lambda x :x.hour)
test['hearing_hour']=test['hearing_date'].apply(lambda x :x.hour)
train['ticket_issued_date']=train['ticket_issued_date'].apply(lambda x :x.toordinal())
train['hearing_date']=train['hearing_date'].apply(lambda x :x.toordinal())
test['ticket_issued_date']=test['ticket_issued_date'].apply(lambda x :x.toordinal())
test['hearing_date']=test['hearing_date'].apply(lambda x :x.toordinal())

In [207]:
latlon=pd.read_csv('latlons.csv')
address=pd.read_csv('addresses.csv')
latlon.dropna(axis=0,inplace=True)
addlatlon=address.merge(latlon,how='inner',on='address')
addlatlon.drop('address',axis=1,inplace=True)
train.drop(['violation_street_number', 'violation_street_name', 'violation_zip_code'],axis=1,inplace=True)
test.drop(['violation_street_number', 'violation_street_name', 'violation_zip_code'],axis=1,inplace=True)
train=train.merge(addlatlon, how='left',on='ticket_id')
test=test.merge(addlatlon,how='left',on='ticket_id')
train[['lat','lon']]=train[['lat','lon']].fillna(value=0)
test[['lat','lon']]=test[['lat','lon']].fillna(value=0)

In [208]:
mailing=['mailing_address_str_number', 'mailing_address_str_name', 'city','state', 'zip_code', 'non_us_str_code', 'country']
train.drop(mailing,axis=1,inplace=True)
test.drop(mailing,axis=1,inplace=True)

In [209]:
#extracting label from dateset
y_train=train['compliance']
#preparing answer series with name and indexes
answer=pd.Series(index=test['ticket_id'],dtype='float32',name='compliance')
#droping features unavailable in test data
train.drop(['payment_amount','payment_date','payment_status','balance_due','collection_status','compliance','compliance_detail'],axis=1,inplace=True)
    

In [219]:
scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(train)
X_test_scaled = scaler.transform(test)

In [ ]:
clf = MLPClassifier(hidden_layer_sizes = [100, 100], alpha = 5.0,
                   random_state = 0, solver='lbfgs').fit(X_train_scaled, y_train)

In [ ]:
proba_test=clf.predict_proba(test)

In [218]:
test[['lat','lon']]=test[['lat','lon']].fillna(value=0)

In [220]:
test.isnull().any()

ticket_id                False
agency_name              False
inspector_name           False
violator_name            False
ticket_issued_date       False
hearing_date             False
violation_code           False
violation_description    False
disposition              False
fine_amount              False
admin_fee                False
state_fee                False
late_fee                 False
discount_amount          False
clean_up_cost            False
judgment_amount          False
grafitti_status          False
ticket_hour              False
hearing_hour             False
lat                      False
lon                      False
dtype: bool

In [216]:
train.dtypes

ticket_id                  int64
agency_name                int64
inspector_name             int64
violator_name              int64
ticket_issued_date         int64
hearing_date               int64
violation_code             int64
violation_description      int64
disposition                int64
fine_amount              float64
admin_fee                float64
state_fee                float64
late_fee                 float64
discount_amount          float64
clean_up_cost            float64
judgment_amount          float64
grafitti_status            int64
ticket_hour                int64
hearing_hour               int64
lat                      float64
lon                      float64
dtype: object

In [38]:
address.head(5)

,ticket_id,address
0,22056,"2900 tyler, Detroit MI"
1,27586,"4311 central, Detroit MI"
2,22062,"1449 longfellow, Detroit MI"
3,22084,"1441 longfellow, Detroit MI"
4,22093,"2449 churchill, Detroit MI"


In [39]:
address.loc[address['address']=='2900 tyler, Detroit MI']

,ticket_id,address
0,22056,"2900 tyler, Detroit MI"
53754,77242,"2900 tyler, Detroit MI"
53803,77243,"2900 tyler, Detroit MI"
78764,103945,"2900 tyler, Detroit MI"
111967,138219,"2900 tyler, Detroit MI"
148274,177558,"2900 tyler, Detroit MI"


In [40]:
address.loc[address['address']=='2900 tyler, Detroit MI']['ticket_id'].as_matrix()

array([ 22056,  77242,  77243, 103945, 138219, 177558])

In [41]:
address.loc[address['ticket_id']==285530 ]['address'].iloc[0]

'3408 beatrice, Detroit MI'

In [42]:
lat=latlon.loc[latlon['address']=='2900 tyler, Detroit MI']['lat'].iloc[0]
lat

42.390729

In [44]:
latlon.head(5)

,address,lat,lon
0,"4300 rosa parks blvd, Detroit MI 48208",42.346169,-83.079962
1,"14512 sussex, Detroit MI",42.394657,-83.194265
2,"3456 garland, Detroit MI",42.373779,-82.986228
3,"5787 wayburn, Detroit MI",42.403342,-82.957805
4,"5766 haverhill, Detroit MI",42.407255,-82.946295


In [75]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
pca = PCA(n_components=1)
latlon.dropna(axis=0,inplace=True)
scaler = StandardScaler()

X_PCA=latlon[['lat','lon']].as_matrix()
X_PCA_scaled = scaler.fit_transform(X_PCA)
llpca=pca.fit_transform(X_PCA)
llpca

array([[-0.02916424],
       [ 0.08185844],
       [-0.12445089],
       ..., 
       [-0.16954946],
       [ 0.04725764],
       [ 0.11970123]])

In [15]:
addlatlon=address.merge(latlon,how='inner',on='address')

In [16]:
addlatlon.head(5)

,ticket_id,address,lat,lon
0,22056,"2900 tyler, Detroit MI",42.390729,83.124268
1,77242,"2900 tyler, Detroit MI",42.390729,83.124268
2,77243,"2900 tyler, Detroit MI",42.390729,83.124268
3,103945,"2900 tyler, Detroit MI",42.390729,83.124268
4,138219,"2900 tyler, Detroit MI",42.390729,83.124268


In [18]:
train[train['ticket_id']==28204]

,ticket_id,agency_name,inspector_name,violator_name,violation_street_number,violation_street_name,violation_zip_code,mailing_address_str_number,mailing_address_str_name,city,...,clean_up_cost,judgment_amount,payment_amount,balance_due,payment_date,payment_status,collection_status,grafitti_status,compliance_detail,compliance
9,28204,"Buildings, Safety Engineering & Env Department","Williams, Darrin","Inc, Nanno",15307.0,SEVEN MILE,NaN,1537.0,E. Seven Mile,Detroit,...,0.0,855.0,0.0,855.0,NaN,NO PAYMENT APPLIED,NaN,NaN,non-compliant by no payment,0.0


In [19]:
train.head(5)

,ticket_id,agency_name,inspector_name,violator_name,violation_street_number,violation_street_name,violation_zip_code,mailing_address_str_number,mailing_address_str_name,city,...,clean_up_cost,judgment_amount,payment_amount,balance_due,payment_date,payment_status,collection_status,grafitti_status,compliance_detail,compliance
0,22056,"Buildings, Safety Engineering & Env Department","Sims, Martinzie","INVESTMENT INC., MIDWEST MORTGAGE",2900.0,TYLER,NaN,3.0,S. WICKER,CHICAGO,...,0.0,305.0,0.0,305.0,NaN,NO PAYMENT APPLIED,NaN,NaN,non-compliant by no payment,0.0
1,27586,"Buildings, Safety Engineering & Env Department","Williams, Darrin","Michigan, Covenant House",4311.0,CENTRAL,NaN,2959.0,Martin Luther King,Detroit,...,0.0,855.0,780.0,75.0,2005-06-02 00:00:00,PAID IN FULL,NaN,NaN,compliant by late payment within 1 month,1.0
2,22062,"Buildings, Safety Engineering & Env Department","Sims, Martinzie","SANDERS, DERRON",1449.0,LONGFELLOW,NaN,23658.0,P.O. BOX,DETROIT,...,0.0,0.0,0.0,0.0,NaN,NO PAYMENT APPLIED,NaN,NaN,not responsible by disposition,NaN
3,22084,"Buildings, Safety Engineering & Env Department","Sims, Martinzie","MOROSI, MIKE",1441.0,LONGFELLOW,NaN,5.0,ST. CLAIR,DETROIT,...,0.0,0.0,0.0,0.0,NaN,NO PAYMENT APPLIED,NaN,NaN,not responsible by disposition,NaN
4,22093,"Buildings, Safety Engineering & Env Department","Sims, Martinzie","NATHANIEL, NEAL",2449.0,CHURCHILL,NaN,7449.0,CHURCHILL,DETROIT,...,0.0,0.0,0.0,0.0,NaN,NO PAYMENT APPLIED,NaN,NaN,not responsible by disposition,NaN


In [20]:
train.dropna(axis=0,subset=['compliance']).head(5)

,ticket_id,agency_name,inspector_name,violator_name,violation_street_number,violation_street_name,violation_zip_code,mailing_address_str_number,mailing_address_str_name,city,...,clean_up_cost,judgment_amount,payment_amount,balance_due,payment_date,payment_status,collection_status,grafitti_status,compliance_detail,compliance
0,22056,"Buildings, Safety Engineering & Env Department","Sims, Martinzie","INVESTMENT INC., MIDWEST MORTGAGE",2900.0,TYLER,NaN,3.0,S. WICKER,CHICAGO,...,0.0,305.0,0.0,305.0,NaN,NO PAYMENT APPLIED,NaN,NaN,non-compliant by no payment,0.0
1,27586,"Buildings, Safety Engineering & Env Department","Williams, Darrin","Michigan, Covenant House",4311.0,CENTRAL,NaN,2959.0,Martin Luther King,Detroit,...,0.0,855.0,780.0,75.0,2005-06-02 00:00:00,PAID IN FULL,NaN,NaN,compliant by late payment within 1 month,1.0
5,22046,"Buildings, Safety Engineering & Env Department","Sims, Martinzie","KASIMU, UKWELI",6478.0,NORTHFIELD,NaN,2755.0,E. 17TH,LOG BEACH,...,0.0,305.0,0.0,305.0,NaN,NO PAYMENT APPLIED,NaN,NaN,non-compliant by no payment,0.0
6,18738,"Buildings, Safety Engineering & Env Department","Williams, Darrin","Deerwood Development Group Inc, Deer",8027.0,BRENTWOOD,NaN,476.0,Garfield,Clinton,...,0.0,855.0,0.0,855.0,NaN,NO PAYMENT APPLIED,NaN,NaN,non-compliant by no payment,0.0
7,18735,"Buildings, Safety Engineering & Env Department","Williams, Darrin","Rafee Auto Services L.L.C., RAF",8228.0,MT ELLIOTT,NaN,8228.0,Mt. Elliott,Detroit,...,0.0,140.0,0.0,140.0,NaN,NO PAYMENT APPLIED,NaN,NaN,non-compliant by no payment,0.0


In [21]:
train[train['grafitti_status'].notnull()]

,ticket_id,agency_name,inspector_name,violator_name,violation_street_number,violation_street_name,violation_zip_code,mailing_address_str_number,mailing_address_str_name,city,...,clean_up_cost,judgment_amount,payment_amount,balance_due,payment_date,payment_status,collection_status,grafitti_status,compliance_detail,compliance
250296,366178,"Buildings, Safety Engineering & Env Department","Bush, Wesley",MARK JACKSON,8020.0,PURITAN,NaN,251.0,HEYDEN,DETROT,...,0.0,0.0,0.0,0.0,NaN,NO PAYMENT APPLIED,NaN,GRAFFITI TICKET,not responsible by disposition,NaN


In [22]:
test=pd.read_csv('test.csv')

In [23]:
test.head(10)

,ticket_id,agency_name,inspector_name,violator_name,violation_street_number,violation_street_name,violation_zip_code,mailing_address_str_number,mailing_address_str_name,city,...,violation_description,disposition,fine_amount,admin_fee,state_fee,late_fee,discount_amount,clean_up_cost,judgment_amount,grafitti_status
0,284932,Department of Public Works,"Granberry, Aisha B","FLUELLEN, JOHN A",10041.0,ROSEBERRY,NaN,141,ROSEBERRY,DETROIT,...,Failure to secure City or Private solid waste ...,Responsible by Default,200.0,20.0,10.0,20.0,0.0,0.0,250.0,NaN
1,285362,Department of Public Works,"Lusk, Gertrina","WHIGHAM, THELMA",18520.0,EVERGREEN,NaN,19136,GLASTONBURY,DETROIT,...,Allowing bulk solid waste to lie or accumulate...,Responsible by Default,1000.0,20.0,10.0,100.0,0.0,0.0,1130.0,NaN
2,285361,Department of Public Works,"Lusk, Gertrina","WHIGHAM, THELMA",18520.0,EVERGREEN,NaN,19136,GLASTONBURY,DETROIT,...,Improper placement of Courville container betw...,Responsible by Default,100.0,20.0,10.0,10.0,0.0,0.0,140.0,NaN
3,285338,Department of Public Works,"Talbert, Reginald","HARABEDIEN, POPKIN",1835.0,CENTRAL,NaN,2246,NELSON,WOODHAVEN,...,Allowing bulk solid waste to lie or accumulate...,Responsible by Default,200.0,20.0,10.0,20.0,0.0,0.0,250.0,NaN
4,285346,Department of Public Works,"Talbert, Reginald","CORBELL, STANLEY",1700.0,CENTRAL,NaN,3435,MUNGER,LIVONIA,...,Violation of time limit for approved container...,Responsible by Default,100.0,20.0,10.0,10.0,0.0,0.0,140.0,NaN
5,285345,Department of Public Works,"Talbert, Reginald","CORBELL, STANLEY",1700.0,CENTRAL,NaN,3435,MUNGER,LIVONIA,...,Allowing bulk solid waste to lie or accumulate...,Responsible by Default,200.0,20.0,10.0,20.0,0.0,0.0,250.0,NaN
6,285347,Department of Public Works,"Talbert, Reginald","CORBELL, STANLEY",1700.0,CENTRAL,NaN,3435,MUNGER,LIVONIA,...,Inoperable motor vehicle(s) one- or two-family...,Responsible by Default,50.0,20.0,10.0,5.0,0.0,0.0,85.0,NaN
7,285342,Department of Public Works,"Talbert, Reginald","NICKOLA CORPORATION, W & H",1605.0,LIVERNOIS,NaN,1382,WHITEHOUSE CT,ROCHESTER HILLS,...,Allowing bulk solid waste to lie or accumulate...,Responsible by Determination,200.0,20.0,10.0,0.0,0.0,0.0,230.0,NaN
8,285530,Department of Public Works,"Buchanan, Daryl","INTERSTATE INVESTMENT GROUP LL, .",3408.0,BEATRICE,NaN,341,HAMPTON,GILBERT,...,Allowing bulk solid waste to lie or accumulate...,Responsible by Default,1000.0,20.0,10.0,100.0,0.0,0.0,1130.0,NaN
9,284989,Department of Public Works,"Buchanan, Daryl","YAMAN, BATURAY",8040.0,SARENA,NaN,43494,ELLSWORTH # 20,FREMONT,...,Allowing bulk solid waste to lie or accumulate...,Responsible by Default,500.0,20.0,10.0,50.0,0.0,0.0,580.0,NaN


In [25]:
train.drop(['violation_street_number', 'violation_street_name', 'violation_zip_code'],axis=1,inplace=True)
train=train.merge(addlatlon, how='left',on='ticket_id')
train.head(5)

,ticket_id,agency_name,inspector_name,violator_name,mailing_address_str_number,mailing_address_str_name,city,state,zip_code,non_us_str_code,...,balance_due,payment_date,payment_status,collection_status,grafitti_status,compliance_detail,compliance,address,lat,lon
0,22056,"Buildings, Safety Engineering & Env Department","Sims, Martinzie","INVESTMENT INC., MIDWEST MORTGAGE",3.0,S. WICKER,CHICAGO,IL,60606,NaN,...,305.0,NaN,NO PAYMENT APPLIED,NaN,NaN,non-compliant by no payment,0.0,"2900 tyler, Detroit MI",42.390729,83.124268
1,27586,"Buildings, Safety Engineering & Env Department","Williams, Darrin","Michigan, Covenant House",2959.0,Martin Luther King,Detroit,MI,48208,NaN,...,75.0,2005-06-02 00:00:00,PAID IN FULL,NaN,NaN,compliant by late payment within 1 month,1.0,"4311 central, Detroit MI",42.326937,83.135118
2,22062,"Buildings, Safety Engineering & Env Department","Sims, Martinzie","SANDERS, DERRON",23658.0,P.O. BOX,DETROIT,MI,48223,NaN,...,0.0,NaN,NO PAYMENT APPLIED,NaN,NaN,not responsible by disposition,NaN,"1449 longfellow, Detroit MI",42.380516,83.096069
3,22084,"Buildings, Safety Engineering & Env Department","Sims, Martinzie","MOROSI, MIKE",5.0,ST. CLAIR,DETROIT,MI,48214,NaN,...,0.0,NaN,NO PAYMENT APPLIED,NaN,NaN,not responsible by disposition,NaN,"1441 longfellow, Detroit MI",42.380570,83.095919
4,22093,"Buildings, Safety Engineering & Env Department","Sims, Martinzie","NATHANIEL, NEAL",7449.0,CHURCHILL,DETROIT,MI,48206,NaN,...,0.0,NaN,NO PAYMENT APPLIED,NaN,NaN,not responsible by disposition,NaN,"2449 churchill, Detroit MI",42.145257,83.208233


In [27]:
le=LabelEncoder()
train['agency_name']=le.fit_transform(train['agency_name'])
train.head(5)

,ticket_id,agency_name,inspector_name,violator_name,mailing_address_str_number,mailing_address_str_name,city,state,zip_code,non_us_str_code,...,balance_due,payment_date,payment_status,collection_status,grafitti_status,compliance_detail,compliance,address,lat,lon
0,22056,0,"Sims, Martinzie","INVESTMENT INC., MIDWEST MORTGAGE",3.0,S. WICKER,CHICAGO,IL,60606,NaN,...,305.0,NaN,NO PAYMENT APPLIED,NaN,NaN,non-compliant by no payment,0.0,"2900 tyler, Detroit MI",42.390729,83.124268
1,27586,0,"Williams, Darrin","Michigan, Covenant House",2959.0,Martin Luther King,Detroit,MI,48208,NaN,...,75.0,2005-06-02 00:00:00,PAID IN FULL,NaN,NaN,compliant by late payment within 1 month,1.0,"4311 central, Detroit MI",42.326937,83.135118
2,22062,0,"Sims, Martinzie","SANDERS, DERRON",23658.0,P.O. BOX,DETROIT,MI,48223,NaN,...,0.0,NaN,NO PAYMENT APPLIED,NaN,NaN,not responsible by disposition,NaN,"1449 longfellow, Detroit MI",42.380516,83.096069
3,22084,0,"Sims, Martinzie","MOROSI, MIKE",5.0,ST. CLAIR,DETROIT,MI,48214,NaN,...,0.0,NaN,NO PAYMENT APPLIED,NaN,NaN,not responsible by disposition,NaN,"1441 longfellow, Detroit MI",42.380570,83.095919
4,22093,0,"Sims, Martinzie","NATHANIEL, NEAL",7449.0,CHURCHILL,DETROIT,MI,48206,NaN,...,0.0,NaN,NO PAYMENT APPLIED,NaN,NaN,not responsible by disposition,NaN,"2449 churchill, Detroit MI",42.145257,83.208233


In [49]:
train['agency_name'].unique()

array([0, 3, 1, 2, 4])

In [68]:
train.shape

(250306, 34)

In [69]:
test.shape

(61001, 27)

In [99]:
train['violator_name']=train[train['violator_name'].isnull()]['violator_name']='unknown'

In [100]:
train[train['violator_name'].isnull()]

,ticket_id,agency_name,inspector_name,violator_name,violation_street_number,violation_street_name,violation_zip_code,mailing_address_str_number,mailing_address_str_name,city,...,clean_up_cost,judgment_amount,payment_amount,balance_due,payment_date,payment_status,collection_status,grafitti_status,compliance_detail,compliance


In [101]:
colnames=['agency_name','inspector_name','violator_name','violation_description','disposition','grafitti_status']
train[colnames]=train[colnames].fillna(value='unknown')
test[colnames]=test[colnames].fillna(value='unknown')

In [105]:
train[train['grafitti_status'].isnull()]

,ticket_id,agency_name,inspector_name,violator_name,violation_street_number,violation_street_name,violation_zip_code,mailing_address_str_number,mailing_address_str_name,city,...,clean_up_cost,judgment_amount,payment_amount,balance_due,payment_date,payment_status,collection_status,grafitti_status,compliance_detail,compliance


In [177]:
int(str(test['ticket_issued_date'].iloc[0]).split()[1].split(sep=":")[0])

IndexError: list index out of range

In [194]:
traintest['judgment_amount'].isnull().any()

False